In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow 
import keras
import nltk 
#nltk.download()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
intents = json.loads(open('intents.json').read())

In [19]:
for sample in intents['intents']:
  print(sample)
  break

{'tag': 'Can a student who has received a temporary certificate of completion be able to return to education?', 'patterns': ['Can a student who has received a temporary certificate of completion be able to return to education?', 'Can a graduate student who has a temporary certificate return to study?', 'Can a graduate associate who has a temporary certificate return to study?'], 'response': ['If he receives a temporary certificate, he cannot return to study.']}


In [4]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [word for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

In [6]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [7]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [8]:
random.shuffle(training)
training = np.array(training)

<ipython-input-8-cc0bc1dae1db>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [9]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
75/75 [==============================] - 1s 2ms/step - loss: 4.8466 - accuracy: 0.0107
Epoch 2/200
75/75 [==============================] - 0s 2ms/step - loss: 4.7431 - accuracy: 0.0375
Epoch 3/200
75/75 [==============================] - 0s 2ms/step - loss: 4.5573 - accuracy: 0.0670
Epoch 4/200
75/75 [==============================] - 0s 2ms/step - loss: 4.2779 - accuracy: 0.1126
Epoch 5/200
75/75 [==============================] - 0s 2ms/step - loss: 3.9136 - accuracy: 0.1582
Epoch 6/200
75/75 [==============================] - 0s 2ms/step - loss: 3.5802 - accuracy: 0.2172
Epoch 7/200
75/75 [==============================] - 0s 2ms/step - loss: 3.2040 - accuracy: 0.2949
Epoch 8/200
75/75 [==============================] - 0s 2ms/step - loss: 2.9542 - accuracy: 0.3190
Epoch 9/200
75/75 [==============================] - 0s 2ms/step - loss: 2.5896 - accuracy: 0.4102
Epoch 10/200
75/75 [==============================] - 0s 2ms/step - loss: 2.3394 - accuracy: 0.4531
Epoch 11/

In [20]:
model.save('chatbotmodel.h5', hist)

## **Chatbot**

In [21]:
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

In [22]:
def clean_up_sentence(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	return sentence_words

In [23]:
def bag_of_words(sentence):
	sentence_words = clean_up_sentence(sentence)
	bag = [0] * len(words)
	for w in sentence_words:
		for i, word in enumerate(words):
			if word == w:
				bag[i] = 1
	return np.array(bag)

In [24]:
def predict_class(sentence):
	bow = bag_of_words(sentence)
	res = model.predict(np.array([bow]))[0]
	ERROR_THRESHOLD = 0.25
	results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

	results.sort(key=lambda x: x[1], reverse=True)
	return_list = []
	for r in results:
		return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
	return return_list

In [27]:
def get_response(intents_list, intents_json):
	tag = intents_list[0]['intent']
	list_of_intents = intents_json['intents']
	for i in list_of_intents:
		if i['tag'] == tag:
			result = random.choice(i['response'])
			break
	return result

In [ ]:
while True:
	message = input("Enter a message:")
	ints = predict_class(message)
	res = get_response(ints, intents)
	print(res)

Enter a message:Can a student who has received a temporary certificate of completion be able to return to education?
1/1 [==============================] - 0s 127ms/step
If he receives a temporary certificate, he cannot return to study.
Enter a message:Is the passage of time included in the cancellation of the obligation of free education?
1/1 [==============================] - 0s 33ms/step
no
